# 构建检索问答链

我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

### Milvus

In [1]:
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import OllamaEmbeddings
my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="dengcao/Qwen3-Embedding-0.6B:F16")

# Milvus 连接参数
vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="ZXVMAXS",  # Milvus 集合名称
        connection_args={
            "host": "129.201.70.35",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
    )

/tmp/ipykernel_1270322/1261815211.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="dengcao/Qwen3-Embedding-0.6B:F16")
/tmp/ipykernel_1270322/1261815211.py:6: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-milvus package and should be used instead. To use it run `pip install -U :class:`~langchain-milvus` and import as `from :class:`~langchain_milvus import MilvusVectorStore``.
  vectordb = Milvus(


In [2]:
results = vectordb.similarity_search(query="什么是vmax的上网日志系统？", k=2)
results

[Document(metadata={'producer': 'Apache FOP Version 2.6', 'creator': 'DITA Open Toolkit', 'creationdate': '2023-05-23T21:45:33+08:00', 'source': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', 'file_path': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', 'total_pages': 29, 'format': 'PDF 1.4', 'title': '目录', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20230523214533+08'00'", 'page': 13, 'pk': 458694506400191703}, page_content='4\xa0功能本章包含如下主题：\uf06c上网日志保存\n10\n\uf06c上网日志查询\n10\n\uf06c上网日志批量导入查询\n11\n\uf06c日志管理\n11\n\uf06c账号管理\n11\n\uf06c角色管理\n12\n\uf06c资源监控\n12\n\uf06c告警管理\n12\n\uf06c省级网关对接\n12\n\uf06c拨测结果自动比对功能\n12\n\uf06cNAT日志入库功能\n13\n\uf06c北向接口\n13\n\uf06c上网日志历史查询\n13\n\uf06c云化上网日志XDR查询\n13\n以下介绍ZXVMAX-S的主要的功能。\n4.1\xa0上网日志保存\uf06c支持使用Gbase数据库或HDFS保存上网日志。\uf06c上网日志保存时间可配置，最短保存7天时间，最长可保存一年时间。\uf06c支持自动清理超过保存时间的上网日志。\n4.2\xa0上网日志查询可通过web界面指定查询条件，查询用户上网日志。支持的查询条件：\uf06c时间范围+公网IP

## 2. 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [30]:
from langchain_community.llms import Ollama

my_llm = Ollama(base_url='http://localhost:11434', model='qwen3:8B', temperature=0.1)

# my_llm.invoke("你好")

/tmp/ipykernel_1272557/2051099669.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  my_llm = Ollama(base_url='http://localhost:11434', model='qwen3:8B', temperature=0.1)


## 3. 构建检索问答链

prompts

In [25]:
from langchain.prompts import PromptTemplate

template = """你是VMAX运维助手，使用以下上下文来回答问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


#### 创建一个基于模板的检索链： 基础检索版本

In [6]:
from langchain.chains import RetrievalQA

# 基础检索
base_retriever = vectordb.as_retriever(search_kwargs={"k": 10})
base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    metadata_filter={"source": "../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（5GC业务）.pdf"}  # 元数据过滤
)

qa_chain = RetrievalQA.from_chain_type(my_llm,
                                       retriever=base_retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


#### 创建一个基于模板的检索链： 混合检索版本

混合检索

Milvus自带混合检索功能，
数据写入向量数据库流程需要修改

参考： https://milvus.io/docs/zh/milvus_hybrid_search_retriever.md

修改147-156行： 

In [1]:
# 如果使用Milvus的混合检索
            # vectordb = Milvus.from_documents(
            # documents=batch_docs,
            # embedding=my_emb,
            # builtin_function=BM25BuiltInFunction(),
            # vector_field=["dense", "sparse"],
            # collection_name="ZXVMAXS2",
            # drop_old=False,
            # connection_args=connection_args,
            # consistency_level="Strong",
            # )       

In [26]:
from langchain_milvus import Milvus, BM25BuiltInFunction
from langchain_community.embeddings import OllamaEmbeddings
my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="dengcao/Qwen3-Embedding-0.6B:F16")

# Milvus 连接参数
vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="ZXVMAXS3",  # Milvus 集合名称
        connection_args={
            "host": "129.201.70.35",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
        builtin_function=BM25BuiltInFunction(),
        vector_field=["dense", "sparse"]
    )

2025-06-19 17:40:55,964 [DEBUG][_create_connection]: Created new connection using: 2b3eb9db3ea94e1b96e6a42c058d3df1 (async_milvus_client.py:599)


定义检索器，从向量数据库检索与问题相关的文档片段

In [27]:
retriever = vectordb.as_retriever()

将检索到的多个文档合并为单一字符串，作为上下文的输入

In [28]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

​​输入处理​​：  
RunnablePassthrough()直接传递用户问题到question字段。  
retriever | format_docs：检索文档并格式化为字符串，作为context。  
​​提示生成​​：prompt将context和question填充到模板中。  
​​模型调用​​：llm生成回答。  
​​输出解析​​：StrOutputParser将模型输出转为字符串。  

In [31]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | QA_CHAIN_PROMPT 
    | my_llm
    | StrOutputParser()
)


In [32]:
query = "ZXVMAX-S上网日志有哪些功能？"
res = rag_chain.invoke(query)
res


'<think>\n好的，用户问的是ZXVMAX-S上网日志有哪些功能。我需要先回顾提供的上下文内容，看看有没有提到相关功能。\n\n首先，上下文里提到ZXVMAX-S端到端系统有四个主要功能：用户分析、网络分析、专题分析和评估优化。还提到支持XDR查询、实时保障、多维分析、会话跟踪等功能。这些可能和上网日志相关。\n\n然后，在产品特点部分，分层架构和可靠性设计有提到，但可能不直接涉及日志功能。需要更仔细看是否有提到日志相关的功能。\n\n在产品定位与特点里，提到支持XDR查询，这可能和日志数据的查询有关。另外，实时分析、周期任务、数据自定义查询或分析，这些可能包括日志的处理。还有事件详细信息的呈现，可能涉及日志记录。\n\n不过，用户的问题是关于“上网日志”的具体功能，而上下文中并没有明确提到“上网日志”这个术语。可能需要推断。例如，XDR查询可能涉及用户数据记录，会话跟踪可能与日志相关，实时保障可能包括日志监控。但如果没有明确说明，就不能编造答案。\n\n因此，正确的做法是根据现有信息，指出可能相关的功能，如XDR查询、实时分析、会话跟踪等，但明确说明没有直接提到上网日志的具体功能。最后按照要求，回答后加上“谢谢你的提问！”。\n</think>\n\n根据提供的上下文，ZXVMAX-S系统支持XDR查询、实时保障、多维分析、会话跟踪等功能，这些功能可能与上网日志相关，但文档中未明确提及“上网日志”的具体功能。因此，无法直接回答该问题。\n\n谢谢你的提问！'

#### 加入ReRanker进行精排

第一阶段检索结束后，我们需要对候选数据重新排名，以获得更好的结果。您可以根据自己的要求选择加权排名器（WeightedRanker）或重新 排名 器（RRFRanker）。

WeightedRanker/RRF：是一种多路召回融合算法，通过加权融合不同检索路径（如向量搜索、关键词搜索）的排名结果，提升召回多样性。但它仅基于排名位置计算分数，不深入分析语义相关性。

Cohere等 Rerank：是基于深度学习的交叉编码器（Cross-Encoder），直接计算查询与文档的语义匹配分数，能更精准地识别上下文关联性，尤其适合处理复杂语义或多义词问题。

所以可以进一步进行Reranker

In [ ]:
from langchain.prompts import PromptTemplate

template = """你是VMAX运维助手，使用以下上下文来回答问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)
